In [37]:
import os 

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import IPython.display as ipd

import librosa
import librosa.display
import pickle

import warnings
warnings.filterwarnings('ignore')

### Load Saved Model

In [38]:
rf = pickle.load(open("../Saved_Models/model.obj", "rb"))
encoder = pickle.load(open("../Saved_Models/encoder.obj", "rb"))

### Get Full File Path from user

In [51]:
#"/Users/vedantterkar/Desktop/Proj/*"
audio_file = input("Enter Full Audio File Path: ")

Enter Full Audio File Path: /Users/vedantterkar/Desktop/Proj/D.mp3


### Listen to File

In [52]:
ipd.Audio(audio_file) 

### Extract Features

In [53]:
audio_clips = [audio_file]

m = {}
m["chroma"], m["spec_cnt"], m["spec_bw"], m["roll_off"] = [], [], [], []
m["zcr"] = []

for i in range(20):
    m["mfcc_"+str(i)] = []


for song in audio_clips:
    x, sr = librosa.load(song, mono=True, offset=0)
    m["chroma"].append(np.mean(librosa.feature.chroma_stft(y=x, sr=sr)))
    m["spec_cnt"].append(np.mean(librosa.feature.spectral_centroid(y=x, sr=sr)))
    m["spec_bw"].append(np.mean(librosa.feature.spectral_bandwidth(y=x, sr=sr)))
    m["roll_off"].append(np.mean(librosa.feature.spectral_rolloff(y=x, sr=sr)))
    m["zcr"].append(np.mean(librosa.feature.zero_crossing_rate(x)))
    mfcc = librosa.feature.mfcc(y=x, sr=sr)
    c = 0
    for e in mfcc:
        m["mfcc_"+str(c)].append(np.mean(e))
        c += 1
    
import pandas as pd
dataFrame = pd.DataFrame(data=m, columns=sorted(list(m.keys()), reverse = True))
dataFrame

,zcr,spec_cnt,spec_bw,roll_off,mfcc_9,mfcc_8,mfcc_7,mfcc_6,mfcc_5,mfcc_4,...,mfcc_16,mfcc_15,mfcc_14,mfcc_13,mfcc_12,mfcc_11,mfcc_10,mfcc_1,mfcc_0,chroma
0,0.107925,2291.929919,2250.978253,4548.803656,5.16437,-8.017694,6.890228,-4.596979,5.799175,4.648535,...,-6.312048,4.657134,-3.853938,5.889839,-1.595478,7.657365,-0.31389,100.618614,-33.961792,0.396736


In [54]:
pred = rf.predict(dataFrame)
pred[0], encoder.inverse_transform(pred)[0]

(3, 'disco')